<a href="https://colab.research.google.com/github/Jhansipothabattula/Machine_Learning/blob/main/Day58.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building CNN Architectures with PyTorch

**Building CNN Architectures in PyTorch Using the nn Module**

## Key Steps

### 1. Define a Model
- Use **torch.nn.Module** to build CNN layers like:
  - convolutional layers
  - pooling layers
  - fully connected layers

### 2. Forward Pass
- Define how input flows through the layers to produce output

### 3. Model Summary
- Inspect the structure and learnable parameters


# Training and Evaluating CNNs in PyTorch

## Training
- Perform forward and backward passes
- Calculate loss
- Update weights using an optimizer

## Evaluation
- Test the model on unseen data
- Compute metrics like accuracy and loss


# Experimenting with CNN Model Design and Tuning Hyperparameters

## Experimentation Areas

### 1. Layer Depth
- Add or remove convolutional and pooling layers to observe the impact

### 2. Filter Size
- Experiment with kernel sizes (e.g., 3 × 3, 5 × 5)

### 3. Learning Rate
- Adjust the learning rate to improve convergence speed and accuracy


**Objective**
- Build, train, evaluate and experiment with CNNs for CIFAR-10 classification using PyTorch

In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Define Transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root="./data",train=True,download=True,transform=transform)

test_dataset = datasets.CIFAR10(root="./data",train=False,download=True,transform=transform)

# Create Data Loaders
train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)

test_loader = DataLoader(test_dataset,batch_size=64,shuffle=False)

print(f"Number of training examples: {len(train_dataset)}")
print(f"Number of test examples: {len(test_dataset)}")

# Define a CNN model:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
      x = self.pool(torch.relu(self.conv1(x)))
      x = self.pool(torch.relu(self.conv2(x)))
      x = x.view(-1, 16 * 5 * 5)
      x = torch.relu(self.fc1(x))
      x = torch.relu(self.fc2(x))
      x = self.fc3(x)
      return x
model = SimpleCNN()
print(model)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training Loop
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0

        for images, labels in train_loader:
            # Zero gradient
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(images)

            # Calculate Loss
            loss = criterion(outputs, labels)

            # Backward Pass
            loss.backward()

            # Update weights
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

train_model(model, train_loader, criterion, optimizer)

# Evaluation loop
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy:{100 * correct / total:.2f}%")

evaluate_model(model, test_loader)

Number of training examples: 50000
Number of test examples: 10000
SimpleCNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
Epoch [1/10], Loss: 2.3044
Epoch [2/10], Loss: 2.3034
Epoch [3/10], Loss: 2.3023
Epoch [4/10], Loss: 2.3013
Epoch [5/10], Loss: 2.3000
Epoch [6/10], Loss: 2.2985
Epoch [7/10], Loss: 2.2964
Epoch [8/10], Loss: 2.2935
Epoch [9/10], Loss: 2.2889
Epoch [10/10], Loss: 2.2817
Test Accuracy:16.27%
